In [1]:
import re
import pandas as pd

In [3]:
file = open("/content/hvd.hn6k5w.norm.txt", "r") # replace filename
doc = file.read()
file.close()
# takes the sentence before and after a <pb>
page_bounds = re.findall(r'[.!?]+[^!?.]*<pb>[^!?.]*[.!?]+', doc)
# gets rid of the end punctuation(s) at the start of the text that the above regex captures
page_bounds_new = [re.sub(r'^[.!?]+', '', text) for text in page_bounds]

In [ ]:
# Load model directly
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForCausalLM
from vllm import LLM, SamplingParams

model = LLM("PleIAs/OCRonos", dtype="float16", max_model_len=8128)

In [ ]:
from tqdm import tqdm

# OCRonos helper function, taken from app.py in the OCRonos app in huggingface
def correct_ocr(user_message):
  sampling_params = SamplingParams(temperature=0.9, top_p=0.95, max_tokens=4000, presence_penalty=0, stop=["#END#"])
  prompt_template = f"### TEXT ###\n{user_message}\n\n### CORRECTION ###\n"
  prompts = [prompt_template]
  outputs = model.generate(prompts, sampling_params, use_tqdm=False)
  generated_text = outputs[0].outputs[0].text
  return generated_text

In [ ]:
preds = []
for i in tqdm(range(len(page_bounds_new))):
  pb = page_bounds_new[i]
  result = correct_ocr(pb)
  preds.append(result)

In [ ]:
# # save to csv
# df2 = pd.DataFrame(preds)
# df2.to_csv("harvard_preds2.csv")